In [1]:
import ast
import pandas as pd
df = pd.read_csv('/content/split_embedqa.csv')
df.head()

,Context,Questions,Answers,Embedding
0,Berdasarkan barang bukti berikut: 1 (satu) pla...,Berikut adalah 5 pertanyaan dalam konteks huku...,Berikut adalah jawaban atas 5 pertanyaan dalam...,"[-0.0228680782020092, -0.4772705137729645, -0...."
1,Berdasarkan barang bukti berikut: 1 (satu) pla...,NaN,1. Jenis narkotika yang ditemukan dalam plasti...,"[0.04503834620118141, -0.12067674845457077, -0..."
2,Berdasarkan barang bukti berikut: 1 (satu) pla...,1. Apakah jenis narkotika yang ditemukan dalam...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."
3,Berdasarkan barang bukti berikut: 1 (satu) pla...,2. Berapa lama pidana penjara yang dijatuhkan ...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."
4,Berdasarkan barang bukti berikut: 1 (satu) pla...,3. Apakah hukuman denda yang dijatuhkan kepada...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."


In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 2.5 MB/s eta 0:00:00


In [3]:
import os
from groq import Groq
from sentence_transformers import SentenceTransformer

In [4]:
os.environ["GROQ_API_KEY"] = "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"

In [5]:
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

def embedding_query(query):
    try:
        embedding = embedding_model.encode(query)
        return embedding
    except Exception as e:
        print(f"An error occurred during embedding generation: {e}")

def get_answer(query):
    try:
        response = Groq.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query}
            ],
            max_tokens=150
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"An error occurred during answer generation: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from scipy.spatial.distance import cosine
import pandas as pd

def rank_distances(query_embedding, embeddings_list):
    distances = [(i, cosine(query_embedding, ast.literal_eval(embedding))) for i, embedding in enumerate(embeddings_list)]
    return sorted(distances, key=lambda x: x[1])

query = "Apa saja barang bukti yang ditemukan dan dijadikan dasar dalam putusan narkotika?"
query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
top_5_df = pd.DataFrame(top_5_results)
top_5_df

,No,Rank,Answer
0,1,0.261456,Berikut adalah jawaban atas 5 pertanyaan dalam...
1,2,0.262156,Berikut jawaban pertanyaan dalam konteks hukum...
2,3,0.264474,3. Berat bersih narkotika jenis Sabu yang dite...
3,4,0.283026,3. Pidana penjara yang dijatuhkan kepada Terda...
4,5,0.284648,"1. Ya, Terdakwa Dedek Syahputra dinyatakan ber..."
